## Importing Libraries

In [ ]:
#!pip install altair
import altair as alt 
# to plot the rankings 
import matplotlib.pyplot as plt 
# to plot the x and y maps
import numpy as np 
# to numerical calculation
import pandas as pd 
# to read dataframes
from sklearn.cluster import KMeans ,AgglomerativeClustering,Birch 
# we usr Brich here
# to apply clustering
from sklearn.preprocessing import StandardScaler 
# pre process before cluster
import warnings
warnings.filterwarnings("ignore") 
# to remove extra prints
from sklearn.decomposition import PCA
import plotly.express as px

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

## Reading the datasets

In [ ]:
List_Of_Players=pd.read_csv("LisOFPlayers18-19.csv") 
#Reading Players Data
Events_data=pd.read_csv("EventsUpdated.csv") 
#Reading Events data
Events_data=Events_data.drop(['Unnamed: 0'],axis=1)
#Removing Extra Columns
List_Of_Players=List_Of_Players.drop(['Unnamed: 0'],axis=1) 
Events_data['game_id']=Events_data['game_id'].astype(int) 
# So Both Dataframes merge correctly 
#Data type conversion should be the same in both Datasets in the next steps it's going to be merged based on a game ID.

## Merging the Datasets

In [ ]:
Pre_Processing_step_dataframe = pd.merge(List_Of_Players, Events_data,  how='left', left_on=['player_id','GameID'], right_on = ['player_id','game_id']) 
#Merge both datasets based on GameID and player_ID

### Pre Processing

In [ ]:
Pre_Processing_step_dataframe=Pre_Processing_step_dataframe[Pre_Processing_step_dataframe['279'].isna()] 
# Remove Kick Off
Pre_Processing_step_dataframe=Pre_Processing_step_dataframe[Pre_Processing_step_dataframe['position']!='Goalkeeper']
Pre_Processing_step_dataframe=Pre_Processing_step_dataframe[Pre_Processing_step_dataframe['6']!=1] 
#Remove Corner taken.
Pre_Processing_step_dataframe=Pre_Processing_step_dataframe[Pre_Processing_step_dataframe['2']!=1] 
#Remove Offside pass
Pre_Processing_step_dataframe=Pre_Processing_step_dataframe[Pre_Processing_step_dataframe['5']!=1] 
#Remove Free kick taken
Pre_Processing_step_dataframe=Pre_Processing_step_dataframe[Pre_Processing_step_dataframe['107']!=1] 
#Remove Throw-in
Pre_Processing_step_dataframe = Pre_Processing_step_dataframe[Pre_Processing_step_dataframe['x'].notna()] 
#Dropping rows where location is empty
Pre_Processing_step_dataframe.head(5) 
#Shows first 5 rows

In [ ]:
Pre_Processing_step_dataframe=Pre_Processing_step_dataframe.reset_index(drop=True)
# resetting index in case it is not
Pre_Processing_step_dataframe['Team_Name']=Pre_Processing_step_dataframe['team_id']==Pre_Processing_step_dataframe['home_team_id']
# checking whther player belong to home team
Pre_Processing_step_dataframe['Team_Name'] = np.where(Pre_Processing_step_dataframe['Team_Name'] == True,Pre_Processing_step_dataframe['home_team_name'], Pre_Processing_step_dataframe['away_team_name'])
# getting team name for player

## Shots_dataframe and Assists

In [ ]:
Pre_Processing_step_dataframe=Pre_Processing_step_dataframe.drop_duplicates() 
#Drop DPre_Processing_step_dataframelicates
Shots_dataframe=Pre_Processing_step_dataframe[Pre_Processing_step_dataframe['type_id'].isin([13,14,15,16])]
# Selection of Qualifiers
Assists=Shots_dataframe[~Shots_dataframe['55'].isna()]
# Qualifiers that are assists
Assists=Assists.reset_index(drop=True)
#  Resetting Index
Assists_dataframe_cleaned=Assists.dropna(axis=1)
# Drop Extra Columns

In [ ]:
Assists_dataframe_cleaned

## Passes

In [ ]:
Pre_Processing_step_dataframe['1']=Pre_Processing_step_dataframe['1'].fillna(0)
# Completing the 1st Events
Pre_Processing_step_dataframe=Pre_Processing_step_dataframe.dropna(axis=1)
#Dataframe_to_merge_for_3_conditions = Dataframe_to_merge_for_3_conditions[Dataframe_to_merge_for_3_conditions['type_id'] == 1]
# Dropping Extra Columns

Pre_Processing_step_dataframe['Shot Assist']=Pre_Processing_step_dataframe['id'].isin(list(Assists_dataframe_cleaned['id']))
# for Checking for Shot Assist Column
Pre_Processing_step_dataframe['Shot Assist']=Pre_Processing_step_dataframe['Shot Assist'].replace({True : 1, False : 0})
# Replacing True False with 0 and 1
Dataframe_to_merge_for_3_conditions=Pre_Processing_step_dataframe

In [ ]:
Dataframe_to_merge_for_3_conditions['Check']=Dataframe_to_merge_for_3_conditions['event_id'].isin(list(Assists_dataframe_cleaned['55']))
# checking if 55 exists

In [ ]:
assists_updated_dataframe = pd.merge(Assists_dataframe_cleaned, Dataframe_to_merge_for_3_conditions,  how='left', left_on=['55','team_id','GameID'], right_on = ['event_id','team_id','GameID']) 
# The condition in which 55==event_id, team_id=team_id and gameid=gameid

In [ ]:
assists_updated_dataframe.shape
# satisfying the clustring

In [ ]:
x = assists_updated_dataframe['x_x']
y = assists_updated_dataframe['y_x']
z = assists_updated_dataframe['GameID']
gridsize = 30 
plt.hexbin(x, y, C=z)   
cb = plt.colorbar()
cb.set_label('GameID')
plt.gcf().set_size_inches(15, 12)
plt.xlabel("X-location")
plt.ylabel("Y-location")
plt.title("Individual Values at all locations 2018-19")
plt.show() 

## To Make Clusters

In [ ]:
Assists_dataframe_cleaned.columns

In [ ]:
hue=Pre_Processing_step_dataframe.dropna()
# for clustring
tocluster=hue[['min','x','y','Shot Assist','1']]
#Clustered on min,sec, x,y ,1  and Assist count column
RemainingCols = hue.drop(['min','x','y','Shot Assist','1'],axis=1) 
#Selection of remaining columns, later used.

In [ ]:
scaler = StandardScaler() 
#Standardize features by removing the mean and scaling to unit variance
scaled_features = scaler.fit_transform(tocluster)
#Fit to data, then transform it.
scaled_features 
#Shows scaled_features

## PCA

In [ ]:
scaled_features

In [ ]:
# Reducing the data from 5 lables to (2 to 4 using PCA)
# Checking how many number of components will be effective

moods_features=scaled_features
storenumber=list()
storeexpl=list()
i=2
while i<5:
    n_components = i
    storenumber.append(i)
    print(n_components)
    pca = PCA(n_components=n_components)
    components = pca.fit_transform(moods_features)

    total_var = pca.explained_variance_ratio_.sum() * 100
    storeexpl.append(total_var)

    labels = {str(i): f"PC {i+1}" for i in range(n_components)}
    labels['color'] = 'Label'

    fig = px.scatter_matrix(
        components,
        dimensions=range(n_components),
        labels=labels,
        title=f'Total Explained Variance: {total_var:.2f}%',
    )
    fig.update_traces(diagonal_visible=False)
    fig.show()
    i=i+1

In [ ]:
# 4 is chosen because 80% INFORMAION IS AVAILABLE
pca = PCA(n_components=4)
principal_components = pca.fit_transform(scaled_features)

In [ ]:
principal_components.shape

In [ ]:
principal_components

In [ ]:
plt.bar(height=pca.explained_variance_ratio_*100,x=[1,2,3,4])
plt.xlabel("Principal Components")
plt.ylabel("Percentage of Explained Variance")
plt.title("Percentage of Explained Variance per Principal Component")
plt.show()

In [ ]:
# Birch will select Number of Optimal Clustes automatically
model = Birch(branching_factor = 50, n_clusters = None, threshold = 1.5)
model.fit(principal_components)
pred = model.predict(principal_components)
pred

In [ ]:
principal_components

In [ ]:
plt.scatter(principal_components[:,0], principal_components[:,1], c=pred, cmap='rainbow', alpha=0.7, edgecolors='b')
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("Principal Component and spread of clusters")
plt.show()

In [ ]:
tocluster['Clusters']=pred
# Assignment of labels
tocluster.head(1)

In [ ]:
tocluster

In [ ]:
tocluster.groupby(['Clusters']).mean() 
#Averages, what is inside each cluster

In [ ]:
Have_VAL=(tocluster['1']==1) | (tocluster['Shot Assist']==1)
#  Check In which clusters have qualifiers and Assists
tocluster['Value_Individual_Pass']=Have_VAL.replace({True : 1, False : 0})
tocluster['Value_Individual_Pass']=tocluster['Value_Individual_Pass']*2
Ranks_data_info=pd.concat([RemainingCols,tocluster],axis=1)
# Combining All Columns

# Expected Goals Assignments based on Clusters

In [ ]:
x = Ranks_data_info['x']
y = Ranks_data_info['y']
z = Ranks_data_info['Value_Individual_Pass']
gridsize = 30 
plt.hexbin(x, y, C=z)   
cb = plt.colorbar()
cb.set_label('Value of Individual Pass')
plt.gcf().set_size_inches(15, 12)
plt.xlabel("X-location")
plt.ylabel("Y-location")
plt.title("Individual Values at all locations 2018-19")
plt.show() 

In [ ]:
RanksInfo=Ranks_data_info.groupby(['x','y'])['Value_Individual_Pass'].mean().reset_index()
x = RanksInfo['x']
y = RanksInfo['y']
z = RanksInfo['Value_Individual_Pass']
gridsize = 30 
plt.hexbin(x, y, C=z)   
cb = plt.colorbar()
cb.set_label('Value of Individual Pass')
plt.gcf().set_size_inches(15, 12)
plt.xlabel("X-location")
plt.ylabel("Y-location")
plt.title("Avearge of Individual Values at all locations 2018-19")
plt.show() 

In [ ]:
RanksInfo

In [ ]:
to_plot=Ranks_data_info[['position','Value_Individual_Pass']]
to_plot.boxplot(by='position')
plt.gcf().set_size_inches(15, 12)
plt.ylabel("Passes Outcome Value")
plt.show() 

## Mean Calculation

In [ ]:
to_plot.groupby("position").mean()

## Standard Deviation

In [ ]:
to_plot.groupby("position").std()

## To Rank players Calculation of Assist Count/ Min Played

In [ ]:
Ranks_data_info=Pre_Processing_step_dataframe.dropna(axis=1)
# Number Of minutes Played
Ranking_dataframe=pd.DataFrame(Ranks_data_info.groupby(['player_name','GameID'])['mins_played'].mean()) 
#Sum of total minutes player played in all matches
Ranking_dataframe=Ranking_dataframe.reset_index()
Ranking_dataframe=pd.DataFrame(Ranking_dataframe.groupby("player_name")['mins_played'].sum())

In [ ]:
Ranking_dataframe

In [ ]:
Ranking_dataframe['Assists count']=assists_updated_dataframe.groupby('player_name_x')['GameID'].count() 
Ranking_dataframe=Ranking_dataframe.dropna()
#Count of assists

In [ ]:
Ranking_dataframe['Assists count per Number of Minutes Played']=(Ranking_dataframe['Assists count']/Ranking_dataframe['mins_played'])
#Calculation of Assists count per Number of Minutes Played of each player

# To Rank players based on Assists count per Number of Minutes Played

In [ ]:
Ranking_dataframe['percent_mark_90'] = np.where(Ranking_dataframe['mins_played'] > np.percentile(Ranking_dataframe['mins_played'], 90), 1, 0)
Ranking_dataframe=Ranking_dataframe[Ranking_dataframe['percent_mark_90']==1]
# selection of players having maximum participation in the game

In [ ]:
Ranking_dataframe.sort_values(["Assists count per Number of Minutes Played"],ascending=False).head(5)

In [ ]:
Ranking_dataframe.sort_values(["Assists count per Number of Minutes Played"],ascending=False).head(5).index

In [ ]:
to_rank=Ranking_dataframe.reset_index()
alt.Chart(to_rank).mark_bar().encode(
    x='Assists count per Number of Minutes Played',
    y=alt.Y('player_name', sort={"encoding": "x", "order": "descending"})
).properties(
    title='Top players for passes considering overall matches'
)

In [ ]:
to_rank.to_csv("RANK_2018-19.csv")

## To Rank Player for Each Position

# Defenders

In [ ]:
defend=Ranks_data_info[Ranks_data_info['position']=='Defender']
defend1=assists_updated_dataframe[assists_updated_dataframe['position_x']=='Defender']
# Number Of minutes defend
# Number Of minutes Played

Ranking_dataframe=pd.DataFrame(defend.groupby(['player_name','GameID'])['mins_played'].mean()) 
#Sum of total minutes player played in all matches
Ranking_dataframe=Ranking_dataframe.reset_index()
Ranking_dataframe=pd.DataFrame(Ranking_dataframe.groupby("player_name")['mins_played'].sum())
Ranking_dataframe['Assists count']=defend1.groupby('player_name_x')['GameID'].count() 
Ranking_dataframe=Ranking_dataframe.dropna()
#Count of assists
#Count of assists
Ranking_dataframe['Assists count per Number of Minutes Played']=(Ranking_dataframe['Assists count']/Ranking_dataframe['mins_played'])
#Calculation of Assists count per Number of Minutes Played of each player
Ranking_dataframe['percent_mark_90'] = np.where(Ranking_dataframe['mins_played'] > np.percentile(Ranking_dataframe['mins_played'], 90), 1, 0)
defend=Ranking_dataframe[Ranking_dataframe['percent_mark_90']==1]
defend=defend.reset_index()
defend.sort_values(["Assists count per Number of Minutes Played"],ascending=False).head(5)

In [ ]:
list(defend.sort_values(["Assists count per Number of Minutes Played"],ascending=False).head(5)['player_name'])

In [ ]:
#to_rank=to_rank.reset_index()
alt.Chart(defend).mark_bar().encode(
    x='Assists count per Number of Minutes Played',
    y=alt.Y('player_name', sort={"encoding": "x", "order": "descending"})
).properties(
    title='Top Defenders '
)

# Strikers

In [ ]:
Striker=Ranks_data_info[Ranks_data_info['position']=='Striker']
Striker1=assists_updated_dataframe[assists_updated_dataframe['position_x']=='Striker']
# Number Of minutes Striker
# Number Of minutes Played
# Number Of minutes Played
Ranking_dataframe=pd.DataFrame(Striker.groupby(['player_name','GameID'])['mins_played'].mean()) #Sum of total minutes player played in all matches
Ranking_dataframe=Ranking_dataframe.reset_index()
Ranking_dataframe=pd.DataFrame(Ranking_dataframe.groupby("player_name")['mins_played'].sum())
Ranking_dataframe['Assists count']=Striker1.groupby('player_name_x')['GameID'].count() #Count of assists
Ranking_dataframe=Ranking_dataframe.dropna()
Ranking_dataframe['Assists count per Number of Minutes Played']=(Ranking_dataframe['Assists count']/Ranking_dataframe['mins_played']) #Calculation of Assists count per Number of Minutes Played  of each player
Ranking_dataframe['percent_mark_90'] = np.where(Ranking_dataframe['mins_played'] > np.percentile(Ranking_dataframe['mins_played'], 90), 1, 0)
Striker=Ranking_dataframe[Ranking_dataframe['percent_mark_90']==1]
Striker=Striker.reset_index()
Striker.sort_values(["Assists count per Number of Minutes Played"],ascending=False).head(5)

In [ ]:
list(Striker.sort_values(["Assists count per Number of Minutes Played"],ascending=False).head(5)['player_name'])

In [ ]:
#to_rank=to_rank.reset_index()
alt.Chart(Striker).mark_bar().encode(
    x='Assists count per Number of Minutes Played',
    y=alt.Y('player_name', sort={"encoding": "x", "order": "descending"})
).properties(
    title='Top Striker '
)

# Midfielder

In [ ]:
MidFielder=Ranks_data_info[Ranks_data_info['position']=='Midfielder']
MidFielder1=assists_updated_dataframe[assists_updated_dataframe['position_x']=='Midfielder']
# Number Of minutes MidFielder
# Number Of minutes Played
# Number Of minutes Played
Ranking_dataframe=pd.DataFrame(MidFielder.groupby(['player_name','GameID'])['mins_played'].mean()) #Sum of total minutes player played in all matches
Ranking_dataframe=Ranking_dataframe.reset_index()
Ranking_dataframe=pd.DataFrame(Ranking_dataframe.groupby("player_name")['mins_played'].sum())
Ranking_dataframe['Assists count']=MidFielder1.groupby('player_name_x')['GameID'].count() #Count of assists
Ranking_dataframe=Ranking_dataframe.dropna()
Ranking_dataframe['Assists count per Number of Minutes Played']=(Ranking_dataframe['Assists count']/Ranking_dataframe['mins_played']) #Calculation of Assists count per Number of Minutes Played  of each player
Ranking_dataframe['percent_mark_90'] = np.where(Ranking_dataframe['mins_played'] > np.percentile(Ranking_dataframe['mins_played'], 90), 1, 0)
MidFielder=Ranking_dataframe[Ranking_dataframe['percent_mark_90']==1]
MidFielder=MidFielder.reset_index()
MidFielder.sort_values(["Assists count per Number of Minutes Played"],ascending=False).head(5)

In [ ]:
list(MidFielder.sort_values(["Assists count per Number of Minutes Played"],ascending=False).head(5)['player_name'])

In [ ]:
#to_rank=to_rank.reset_index()
alt.Chart(MidFielder).mark_bar().encode(
    x='Assists count per Number of Minutes Played',
    y=alt.Y('player_name', sort={"encoding": "x", "order": "descending"})
).properties(
    title='Top Midfielder '
)

# Top 5 Players with best passings In each team

In [ ]:
Teams=list(Ranks_data_info['Team_Name'].unique())

In [ ]:
# Number Of minutes Played
Ranking_dataframe=pd.DataFrame(Ranks_data_info.groupby(['player_name','Team_Name','GameID'])['mins_played'].mean()) #Sum of total minutes player played in all matches
Ranking_dataframe=Ranking_dataframe.reset_index()
Ranking_dataframe=pd.DataFrame(Ranking_dataframe.groupby(["player_name","Team_Name"])['mins_played'].sum())

In [ ]:
Ranking_dataframe['Assists count']=assists_updated_dataframe.groupby(['player_name_x','Team_Name_x'])['GameID'].count() #Count of assists
Ranking_dataframe=Ranking_dataframe.dropna()
Ranking_dataframe['Assists count per Number of Minutes Played']=(Ranking_dataframe['Assists count']/Ranking_dataframe['mins_played'])

In [ ]:
Teams_Information=Ranking_dataframe.reset_index()

In [ ]:
Teams_Information

In [ ]:
i=0
while i<len(Teams):
    copy=Teams_Information[Teams_Information['Team_Name']==Teams[i]]
    copy=copy.sort_values(["mins_played"],ascending=False).head(10)
    copy=copy.drop(["Team_Name"],axis=1)
    print("\n Team ", Teams[i])
    print(list(copy.sort_values(["Assists count per Number of Minutes Played"],ascending=False).head(5)['player_name']))
    chart =alt.Chart(copy).mark_bar().encode(
        x='Assists count per Number of Minutes Played',
        y=alt.Y('player_name', sort= {"encoding": "x", "order": "descending"})
    ).properties(
        title='Season 2018-19 Top 10 players for team '+str(Teams[i])
    )
    chart.display()
    i=i+1

In [ ]:
Teams_Information=Teams_Information[Teams_Information['mins_played']>1000]

In [ ]:
TEAMS_RANK=pd.DataFrame(Teams_Information.groupby("Team_Name")['Assists count per Number of Minutes Played'].mean())

In [ ]:
TEAMS_RANK.sort_values("Assists count per Number of Minutes Played",ascending=False)

In [ ]:
TEAMS_RANK.to_csv("Team_Rank2018-19.csv")

# 3D plot (Important Areas)

In [ ]:
assists_updated_dataframe['Relx']=assists_updated_dataframe['x_x'].astype(int)
assists_updated_dataframe['Rely']=assists_updated_dataframe['y_x'].astype(int)
To_plot=assists_updated_dataframe.groupby(['Relx','Rely'])['Check'].count().reset_index()
# you have all the assist locations you know which places on the field are important to pass from.
x=To_plot['Relx']
y=To_plot['Rely']
z=To_plot['Check']
mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter3D(x, y, z, c=z, cmap='Greens', label='Assist Number of Assists in Locations');
ax.legend()

ax.set_xlabel('$X$', fontsize=20, rotation=150)
ax.set_ylabel('$Y$', fontsize=20, rotation=30)
ax.set_zlabel(r'Assist Count', fontsize=10, rotation=60)
ax.yaxis._axinfo['label']['space_factor'] = 3.0

plt.show()

# Passes other then assists and players imporatnce based on passing on those areas

In [ ]:
To_plot.columns=['Relx','Rely','Number of Assists in Location']
Events_data = pd.merge(assists_updated_dataframe, To_plot,  how='left', left_on=['Relx','Rely'], right_on = ['Relx','Rely'])
otherpass=Shots_dataframe[Shots_dataframe['55'].isna()]
# passes other then assists
otherpass['Relx']=otherpass['x'].astype(int)
otherpass['Rely']=otherpass['y'].astype(int)
Events_data = pd.merge(otherpass, To_plot,  how='left', left_on=['Relx','Rely'], right_on = ['Relx','Rely'])
Events_data['Number of Assists in Location']=Events_data['Number of Assists in Location'].fillna(0)
PassesIMport=Events_data.groupby('player_name')['Number of Assists in Location'].sum().reset_index().sort_values("Number of Assists in Location").reset_index(drop=True)

In [ ]:
PassesIMport

In [ ]:
TOP20=PassesIMport.tail(20)
#to_rank=to_rank.reset_index()
alt.Chart(TOP20).mark_bar().encode(
    x='Number of Assists in Location',
    y=alt.Y('player_name', sort={"encoding": "x", "order": "descending"})
).properties(
    title='Top Players based on Area '
)

In [ ]:
Events_data = pd.merge(Teams_Information, PassesIMport,  how='left', left_on=['player_name'], right_on = ['player_name'])

In [ ]:
i=0
while i<len(Teams):
    copy=Events_data[Events_data['Team_Name']==Teams[i]]
    copy=copy.sort_values(["mins_played"],ascending=False).head(10)
    copy=copy.drop(["Team_Name"],axis=1)
    print("\n Team ", Teams[i])
    print(list(copy.sort_values(["Number of Assists in Location"],ascending=False).head(5)['player_name']))
    chart =alt.Chart(copy).mark_bar().encode(
        x='Number of Assists in Location',
        y=alt.Y('player_name', sort= {"encoding": "x", "order": "descending"})
    ).properties(
        title='Season 2018-19 Top 10 players for team '+str(Teams[i])
    )
    chart.display()
    i=i+1

In [ ]:
TEAMS_RANK1=pd.DataFrame(Events_data.groupby("Team_Name")['Number of Assists in Location'].mean())

In [ ]:
TEAMS_RANK1.sort_values("Number of Assists in Location",ascending=False)